### SageMaker self supervised prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
version_tag="202209190430"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:

abstract_trainfile = "s3://{}/self-supervised/train.json".format(bucket)
abstract_testfile= "s3://{}/self-supervised/test.json".format(bucket)
abstract_valfile="s3://{}/self-supervised/val.json".format(bucket)

abstract_trainfile = "s3://{}/self-supervised-fake/3048_767_573/train.json".format(bucket)
abstract_testfile= "s3://{}/self-supervised-fake/3048_767_573/test.json".format(bucket)
abstract_valfile="s3://{}/self-supervised-fake/3048_767_573/val.json".format(bucket)
abstract_fake = "s3://{}/self-supervised-fake/3048_767_573/".format(bucket)

abstract_largescale = "s3://{}/selfsupervisedlargescale/pubmedabstracts/".format(bucket)

eval_file = abstract_fake
filepattern =  "{}/*.json" # "{}/*.tsv" # #  # 

instance_type =  "ml.g4dn.2xlarge" # "ml.p3.2xlarge"  #  #ml.g4dn.2xlarge
instance_count = 1

In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [7]:
training_job="selfsupervised-fake-3048-767-573-bert-f-2022-10-16-05-09-43-840"

In [8]:
s3_model_path = f"s3://aegovan-data/selfsupervised_results/{training_job}/output/model.tar.gz"


### Run  prediction

In [9]:
#s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_{}_{}/".format(job_prefix,date_fmt)
s3_output_predictions = "s3://aegovan-data/selfsupervised_chemprot/predictions_{}/{}".format(training_job,date_fmt)
s3_input_data = eval_file
s3_data_type="S3Prefix"
usefilter=0
filter_threshold_std=1.0

s3_input_models = s3_model_path
s3_input_vocab = "s3://{}/embeddings/bert/".format(bucket)

In [10]:
s3_input_data, s3_data_type

('s3://aegovan-data/self-supervised-fake/3048_767_573/', 'S3Prefix')

In [11]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                       volume_size_in_gb = 200,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="selfsupervised-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"

input_file_name = s3_input_data.split("/")[-1]

script_processor.run(
        code='../src/inference/chemprot_selfsupervised_batch_predict.py',

        arguments=[
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output,
            "--ensemble", "0",
            "--tokenisor_data_dir", sm_local_input_vocab,           
            "--filter", str(usefilter),
            "--batch", "32",
            "--filterstdthreshold", str(filter_threshold_std),
            "--filepattern",filepattern
        ],

        inputs=[
                ProcessingInput(
                    source=s3_input_data,
                    s3_data_type = s3_data_type,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="ShardedByS3Key"),

            ProcessingInput(
                    source=s3_input_models,
                    destination=sm_local_input_models,
                    s3_data_distribution_type="FullyReplicated"),
            
            ProcessingInput(
                    source=s3_input_vocab,
                    destination=sm_local_input_vocab,
                    s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )





Job Name:  selfsupervised-inference-2022-10-17-02-01-31-261
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/self-supervised-fake/3048_767_573/', 'LocalPath': '/opt/ml/processing/input/data/jsondata', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/selfsupervised_results/selfsupervised-fake-3048-767-573-bert-f-2022-10-16-05-09-43-840/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/data/models', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/embeddings/bert/', 'LocalPath': '/opt/ml/processing/input/data/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3

2022-10-17 02:08:47,722 - inference.predictor - INFO - Completed inference cuda:0
2022-10-17 02:08:47,722 - inference.ensemble_predictor - INFO - Computing average 
2022-10-17 02:08:47,723 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-10-17 02:08:47,730 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-10-17 02:08:47,831 - inference.batch_predict - INFO - Records to write: 3048
2022-10-17 02:08:47,831 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/train.json.json
2022-10-17 02:08:47,916 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/train.json
2022-10-17 02:08:47,922 - i